# **99Bikes**

## Preprocessing

### Importing Libraries and Data

In [ ]:
# import file to clean and read into pandas dataframe
import pandas as pd

from google.colab import files
uploaded = files.upload()

## read Excel file into pandas dataframe
file_name_raw = '99Bikers_Raw_data.xlsx'
xl_file_raw = pd.ExcelFile(file_name_raw)

Saving 99Bikers_Raw_data.xlsx to 99Bikers_Raw_data.xlsx


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Create a dictionary containing a DataFrame for every sheet
dfs = {sheet_name: xl_file_raw.parse(sheet_name)
          for sheet_name in xl_file_raw.sheet_names}

<ipython-input-10-4155500e0cc7>:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  dfs = {sheet_name: xl_file_raw.parse(sheet_name)
<ipython-input-10-4155500e0cc7>:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  dfs = {sheet_name: xl_file_raw.parse(sheet_name)


In [ ]:
# create a dataframe for each sheet
#dict_keys(['Transactions', 'NewCustomerList', 'CustomerDemographic', 'CustomerAddress'])
df_transactions = dfs['Transactions']
df_newcustomerlist = dfs['NewCustomerList']
df_customerdemographic = dfs['CustomerDemographic']
df_customeraddress = dfs['CustomerAddress']

### Getting information about the data and Cleaning the data

In [ ]:
# Get info
## print sheet names
print(dfs.keys())

dict_keys(['Transactions', 'NewCustomerList', 'CustomerDemographic', 'CustomerAddress'])


1. Transactions

In [ ]:
print(df_transactions.dtypes)

transaction_id                      int64
product_id                          int64
customer_id                         int64
transaction_date           datetime64[ns]
online_order                      float64
order_status                       object
brand                              object
product_line                       object
product_class                      object
product_size                       object
list_price                        float64
standard_cost                     float64
product_first_sold_date           float64
dtype: object


In [ ]:
# Data cleaning for df_transactions
## remove white space of column names
df_transactions.columns = df_transactions.columns.str.strip()
#fill na
df_transactions.fillna('n/a', inplace=True)

##  standardize column data type
df_transactions['transaction_date'] = pd.to_datetime(df_transactions['transaction_date']).dt.date
df_transactions['online_order'] = df_transactions['online_order'].astype('bool') # need to decide how to deal with missing values/null values
df_transactions['order_status'] = df_transactions['order_status'].astype('category')
df_transactions['brand'] = df_transactions['brand'].astype('string')
df_transactions['product_line'] = df_transactions['product_line'].astype('category')
df_transactions['product_class'] = df_transactions['product_class'].astype('category')
df_transactions['product_size'] = df_transactions['product_size'].astype('category')


In [ ]:
# show infomation of df_transactions after cleaning
print(df_transactions.head(10))

   transaction_id  product_id  customer_id transaction_date  online_order  \
0               1           2         2950       2017-02-25         False   
1               2           3         3120       2017-05-21          True   
2               3          37          402       2017-10-16         False   
3               4          88         3135       2017-08-31         False   
4               5          78          787       2017-10-01          True   
5               6          25         2339       2017-03-08          True   
6               7          22         1542       2017-04-21          True   
7               8          15         2459       2017-07-15         False   
8               9          67         1305       2017-08-10         False   
9              10          12         3262       2017-08-30          True   

  order_status           brand product_line product_class product_size  \
0     Approved           Solex     Standard        medium       medium   
1   

2. Customer Demographic

In [ ]:
# Data inspection
print(df_customerdemographic.keys())
print(df_customerdemographic.isnull().sum())
print(df_customerdemographic.dtypes)
print(df_customerdemographic['gender'].unique())
print(df_customerdemographic['wealth_segment'].unique())
print(df_customerdemographic['deceased_indicator'].unique())
print(df_customerdemographic['owns_car'].unique())

Index(['customer_id', 'first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'default', 'owns_car', 'tenure'],
      dtype='object')
customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
dtype: int64
customer_id                                     int64
first_name                                     object
last_name                                      object
gender

In [ ]:
# Data cleaning for df_customerdemographic
## remove white space of column names
df_customerdemographic.columns = df_customerdemographic.columns.str.strip()

#fill na
df_customerdemographic.fillna('n/a', inplace=True)

# => string
df_customerdemographic['first_name'] = df_customerdemographic['first_name'].astype('string')
df_customerdemographic['last_name'] = df_customerdemographic['last_name'].astype('string')
df_customerdemographic['wealth_segment'] = df_customerdemographic['wealth_segment'].astype('category')
df_customerdemographic['deceased_indicator'] = df_customerdemographic['deceased_indicator'].replace({'N': False, 'Y': True})
df_customerdemographic['owns_car'] = df_customerdemographic['owns_car'].replace({'Yes': True, 'No': False})
df_customerdemographic['DOB'] = pd.to_datetime(df_customerdemographic['DOB'], errors='coerce')
df_customerdemographic.dropna(subset=['DOB'], inplace=True)

## Remove columns that are not needed
if 'default' in df_customerdemographic.columns:
    df_customerdemographic = df_customerdemographic.drop('default', axis=1)
if 'tenure' in df_customerdemographic.columns:
    df_customerdemographic = df_customerdemographic.drop('tenure', axis=1)

# standardize column data type
# Gender: F -> Female, M -> Male, Other -> Unspectified
df_customerdemographic['gender'] = df_customerdemographic['gender'].replace({'F': 'Female', 'M': 'Male', 'Female': 'Female', 'Male': 'Male', 'Femal': 'Female', 'U': 'Unspecified'})


In [ ]:
# Sumarize and show infomation of df_customerdemographic after cleaning

print(df_customerdemographic.info())
print(df_customerdemographic.head())
print(df_customerdemographic.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3913 entries, 0 to 3999
Data columns (total 11 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   customer_id                          3913 non-null   int64         
 1   first_name                           3913 non-null   string        
 2   last_name                            3913 non-null   string        
 3   gender                               3913 non-null   object        
 4   past_3_years_bike_related_purchases  3913 non-null   int64         
 5   DOB                                  3913 non-null   datetime64[ns]
 6   job_title                            3913 non-null   object        
 7   job_industry_category                3913 non-null   object        
 8   wealth_segment                       3913 non-null   category      
 9   deceased_indicator                   3913 non-null   bool          
 10  owns_car    

3. New Customer List

In [ ]:
# Data inspection
print(df_newcustomerlist.keys())
print(df_newcustomerlist.dtypes)
print(df_newcustomerlist.isnull().sum())
print(df_newcustomerlist['gender'].unique())
print(df_newcustomerlist['wealth_segment'].unique())
print(df_newcustomerlist['deceased_indicator'].unique())
print(df_newcustomerlist['owns_car'].unique())
print(df_newcustomerlist['property_valuation'].unique())
print(df_newcustomerlist)

Index(['first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'owns_car', 'tenure', 'address', 'postcode', 'state', 'country',
       'property_valuation', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Rank', 'Value'],
      dtype='object')
first_name                                     object
last_name                                      object
gender                                         object
past_3_years_bike_related_purchases             int64
DOB                                    datetime64[ns]
job_title                                      object
job_industry_category                          object
wealth_segment                                 object
deceased_indicator                             object
owns_car                                       object
tenure                                          int64
a

In [ ]:
# Data cleaning for df_newcustomerlist
df_newcustomerlist.columns = df_newcustomerlist.columns.str.strip()

#fill na
df_newcustomerlist.fillna('n/a', inplace=True)


# drop all unamed columns
# Recheck this part!!
df_newcustomerlist = df_newcustomerlist.loc[:, ~df_newcustomerlist.columns.str.contains('^Unnamed')]

# standardize column data type
df_newcustomerlist['first_name'] = df_newcustomerlist['first_name'].astype('string')
df_newcustomerlist['last_name'] = df_newcustomerlist['last_name'].astype('string')
df_newcustomerlist['DOB'] = pd.to_datetime(df_newcustomerlist['DOB'], errors='coerce')
df_newcustomerlist.dropna(subset=['DOB'], inplace=True)
df_newcustomerlist['job_title'] = df_newcustomerlist['job_title'].astype('string')
df_newcustomerlist['job_industry_category'] = df_newcustomerlist['job_industry_category'].astype('string')
df_newcustomerlist['wealth_segment'] = df_newcustomerlist['wealth_segment'].astype('category')
df_newcustomerlist['deceased_indicator'] = df_newcustomerlist['deceased_indicator'].replace({'N': False, 'Y': True})
df_newcustomerlist['owns_car'] = df_newcustomerlist['owns_car'].replace({'Yes': True, 'No': False})
df_newcustomerlist['address'] = df_newcustomerlist['address'].astype('string')
df_newcustomerlist['postcode'] = df_newcustomerlist['postcode'].astype('int64')
df_newcustomerlist['state'] = df_newcustomerlist['state'].astype('category')
# Gender: F -> Female, M -> Male, Other -> Unspectified
df_newcustomerlist['gender'] = df_newcustomerlist['gender'].replace({'F': 'Female', 'M': 'Male', 'Female': 'Female', 'Male': 'Male', 'Femal': 'Female', 'U': 'Unspecified'})

<ipython-input-24-22a0e488ddfb>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_newcustomerlist['first_name'] = df_newcustomerlist['first_name'].astype('string')
<ipython-input-24-22a0e488ddfb>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_newcustomerlist['last_name'] = df_newcustomerlist['last_name'].astype('string')
<ipython-input-24-22a0e488ddfb>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

4. Customer Address

In [ ]:
# Data inspection
print(df_customeraddress.keys())
print(df_customeraddress.dtypes)
print(df_customeraddress.isnull().sum())
print(df_customeraddress['state'].unique())

Index(['customer_id', 'address', 'postcode', 'state', 'country',
       'property_valuation'],
      dtype='object')
customer_id            int64
address               object
postcode               int64
state                 object
country               object
property_valuation     int64
dtype: object
customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64
['New South Wales' 'QLD' 'VIC' 'NSW' 'Victoria']


In [ ]:
# Data cleaning for df_newcustomerlist
df_newcustomerlist.columns = df_newcustomerlist.columns.str.strip()

#fill na
df_customeraddress.fillna('n/a', inplace=True)

# drop all unamed columns
# Recheck this part!!
df_newcustomerlist = df_newcustomerlist.loc[:, ~df_newcustomerlist.columns.str.contains('^Unnamed')]


df_customeraddress['address'] = df_customeraddress['address'].astype('string')
df_customeraddress['postcode'] = df_customeraddress['postcode'].astype('int64')

df_customeraddress['state'] = df_customeraddress['state'].replace({'New South Wales': 'NSW', 'Victoria': 'VIC', 'QLD': 'QLD', 'NSW': 'NSW'})
df_customeraddress['state'] = df_customeraddress['state'].astype('category')

In [ ]:
# Download

# export DataFrame to Excel file
from google.colab import files
df_transactions.to_excel('output_transactions.xlsx', index=False)
files.download('output_transactions.xlsx')

df_customerdemographic.to_excel('output_customerdemographic.xlsx', index=False)
files.download('output_customerdemographic.xlsx')

df_newcustomerlist.to_excel('output_newcustomerlist.xlsx', index=False)
files.download('output_newcustomerlist.xlsx')

df_customeraddress.to_excel('output_customeraddress.xlsx', index=False)
files.download('output_customeraddress.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>